
# 게임 플레이어 인원수 또는 가격 세일 정보를 가져오는 함수

In [3]:
#
import warnings
warnings.filterwarnings(action='ignore')

### 게임 ID 선정
game_id = 374320

import pandas as pd

# 얼마나 플레이하는가 탐색
def create_sale_user_relation(game_id):

    url_u = 'game_data\{}_u.csv'.format(game_id)
    url_s = 'game_data\{}_p.csv'.format(game_id)

    user_df = pd.read_csv(url_u)
    sale_df = pd.read_csv(url_s)
    user_df['DateTime'] = pd.to_datetime(user_df['DateTime'])
    user_df = user_df.fillna(0)

    user_df = user_df.drop(index=user_df[user_df['DateTime'].dt.minute != 0].index)
    user_df = user_df.drop(index=user_df[user_df['DateTime'].dt.hour != 0].index)
    user_df = user_df.reset_index()

    rev_game = (user_df)

    for item in user_df.select_dtypes(float).columns:
        user_df[item] = user_df[item].apply(int)

    sale_df['Original price'] = 49800 # 최신 정가 가격
    sale_df['Sale Rate'] = round(1 - sale_df['Final price']/sale_df['Original price'],2)
    sale_df['DateTime'] = pd.to_datetime(sale_df['DateTime'])

    # 가격 기간동안 유저수
    newPlayerData = []
    newAvgPlayerData = []
    newTwitchData = []


    day_time = abs(int(str(sale_df['DateTime'].dt.date.iloc[0] - sale_df['DateTime'].dt.date.iloc[1]).split()[0]))

    temp = sale_df['DateTime'].iloc[0]
    newPlayerData.append(round(user_df[user_df['DateTime'] < temp]['Players'].sum()/day_time,4))
    newAvgPlayerData.append(round(user_df[user_df['DateTime'] < temp]['Average Players'].mean(),4))
    newTwitchData.append(round(user_df[user_df['DateTime'] < temp]['Twitch Viewers'].sum()/day_time,4))

    for i in sale_df['DateTime']:
        if i == temp:
            continue
        
        day_time = abs(int(str(i - temp).split()[0]))
        newPlayerData.append(round(user_df[(temp <= user_df['DateTime']) & (user_df['DateTime'] < i)]['Players'].sum()/day_time,4))
        #newAvgPlayerData.append(round(user_df[(temp <= user_df['DateTime']) & (user_df['DateTime'] < i)]['Average Players'].mean(),4))
        #newTwitchData.append(round(user_df[(temp <= user_df['DateTime']) & (user_df['DateTime'] < i)]['Twitch Viewers'].sum()/day_time,4))
        temp = i

    sale_df['Players'] = newPlayerData
    #sale_df['Average Players'] = newAvgPlayerData
    #sale_df['Twitch Viewers'] = newTwitchData
    sale_df = sale_df.dropna()

    for item in sale_df[sale_df['Sale Rate'] < 0].index:
        sale_df['Original price'].iloc[item] = sale_df['Final price'].iloc[item]
        sale_df['Sale Rate'].iloc[item] = 0

    return sale_df.reset_index()

create_sale_user_relation(game_id)


,index,DateTime,Final price,Original price,Sale Rate,Players
0,0,2016-02-15 23:40:11,65000,65000,0.00,4.2500
1,1,2016-03-02 22:50:08,49800,49800,0.00,8.4000
2,2,2016-06-23 21:41:08,37350,49800,0.25,29410.3929
3,3,2016-07-04 18:11:14,49800,49800,0.00,12185.1000
4,4,2016-08-23 17:10:38,33370,49800,0.33,7674.1020
...,...,...,...,...,...,...
76,79,2023-09-07 17:01:28,24900,49800,0.50,7195.6364
77,80,2023-09-11 17:21:36,49800,49800,0.00,8651.2500
78,81,2023-11-21 18:29:16,24900,49800,0.50,6456.5211
79,82,2023-11-28 18:27:05,49800,49800,0.00,12279.1667


---
# 게임 정보를 탐색하여 pandas 로 가져오는 함수

In [2]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

#second_list = [601150,374320,1147560,582010,960781,2512790, 1623730, 1203220, 271590, 1086940, 1203620, 582010, 252490, 1938090, 2252570, 1245620, 289070, 322330, 2195250, 1966720, 359550, 218620, 413150, 1174180, 394360, 221100, 2072450, 105600, 1778820, 1222670, 1238810, 251570, 1091500, 346110, 550, 381210, 227300, 250900, 261550, 108600]
second_list = [1038250]
#second_list = [601150,582010,960781,1203220,271590,1203620,582010,2195250,218620,1222670,381210]

def create_game_info(game_id_list):

    # 데이터 프레임 구성하기
    game_info_columns = [
        'AppID',
        'Name',
        'Developer',
        'Company',
        'price',
        'latest rating',
        'latest rating counter',
        'all rating',
        'all rating counter',
        'release date',
        'category',
        'DLC_Game']

    df = pd.DataFrame(columns=game_info_columns)

    # 게임 인덱스 구성하기
    for game_id in game_id_list:
        print(game_id)
        url = 'https://store.steampowered.com/app/{}/'.format(game_id)

        res = requests.get(url)
        soup = BeautifulSoup(res.text)

        # soup.select() 안에 값을 넣어서 찾아주세요
        # 'AppID'
        rev_game_id = game_id
        # 'Name', 게임 이름
        rev_game_name = (soup.select('#appHubAppName')[0].text)
        # 'Developer', 개발자
        rev_game_dev = (soup.select('#game_highlights > div.rightcol > div > div.glance_ctn_responsive_left > div:nth-child(3) > div.summary.column > a')[0].text)
        # 'Company', 배급사
        company_el = soup.select('#game_highlights > div.rightcol > div > div.glance_ctn_responsive_left > div:nth-child(4) > div.summary.column > a')
        rev_game_company = []
        for company in company_el:
            rev_game_company.append(company.text)
        # # 'price', 현재가격
        try:
            rev_game_price = (soup.select('.game_area_purchase_game > div.game_purchase_action > div > div.game_purchase_price.price')[0].text.strip())
        except:
            try:

#game_area_purchase_section_add_to_cart_281619 > div.game_purchase_action
                #game_area_purchase_section_add_to_cart_281619 > div.game_purchase_action > div > div.discount_block.game_purchase_discount > div.discount_prices > div.discount_original_price
                rev_game_price = (soup.select('.game_area_purchase_game > div.game_purchase_action > div > div.discount_block.game_purchase_discount > div.discount_prices > div.discount_original_price')[0].text.strip())
            except:
                rev_game_price = 0
        
        print('게임가격 : ',rev_game_price)

        # # 'latest rating', 최근 평가
        
        try:
            rev_game_late_rating = (soup.select('#review_histogram_rollup_section > div.user_reviews_summary_bar > div > span.game_review_summary.positive')[0].text)
        except:
            rev_game_late_rating = np.nan
        # # 'latest rating counter', 최근 평가 인원
        try:
            rev_game_late_rating_counter = (soup.select('#review_histogram_recent_section > div.user_reviews_summary_bar > div > span:nth-child(3)')[0].text)
        except:
            rev_game_late_rating_counter = 0
        # # 'all rating', 모든 평가
        try:
            rev_game_all_rating = (soup.select('#review_histogram_rollup_section > div.user_reviews_summary_bar > div > span.game_review_summary')[0].text)
            print("모든평가 : " , rev_game_all_rating)
        except:
            rev_game_all_rating = np.nan
        # # 'all rating counter', 모든 평가 인원
        try:
            rev_game_all_rating_count = (soup.select('#review_histogram_rollup_section > div.user_reviews_summary_bar > div > span:nth-child(3)')[0].text)
        except:
            rev_game_all_rating_count = 0
        # # 'release date', 출시일
        rev_game_release_date = (soup.select('#game_highlights > div.rightcol > div > div.glance_ctn_responsive_left > div.release_date > div.date')[0].text)
        # 'DLC_Game', DLC게임인 경우 '#game_area_purchase > div.game_area_bubble.game_area_dlc_bubble' 이 영역이 생깁니다. 
        # 예) https://store.steampowered.com/app/378649/The_Witcher_3_Wild_Hunt__Hearts_of_Stone/?curator_clanid=6126332

        # 장르
        company_category = soup.select('#genresAndManufacturer > span > a')
        rev_game_category = []
        for category in company_category:
            rev_game_category.append(category.text)

        try: 
            try:
                dlc_ori_id = soup.select('#game_highlights > div.rightcol > div > div.glance_details > p > a')[0]['href']
                rev_game_dlc_root = (''.join(filter(lambda x: x.isdigit(), dlc_ori_id)))
            except:
                rev_game_dlc_root = np.nan
            
            rev_game_matacritic_score = np.nan
        except:
            #print('dlc 아님') # 임시로 넣음 
            rev_game_dlc_root = np.nan
            rev_game_matacritic_score = (soup.select('#game_area_metascore > div.score.high')[0].text.strip())
            pass

        new_list = [
            rev_game_id,rev_game_name,rev_game_dev,rev_game_company,rev_game_price,rev_game_late_rating,rev_game_late_rating_counter,rev_game_all_rating,rev_game_all_rating_count,rev_game_release_date,rev_game_category,rev_game_dlc_root
        ]

        df.reset_index()
        df.loc[len(df)] = new_list

    # 데이터 최종 전처리
        
    try:    
        df['price'] = df['price'].str.replace(',','')
        df['price'] = df['price'].str.replace('₩','')

        df['price'].iloc[df[df['price'] == 'Free'].index] = 0
        df['price'].iloc[df[df['price'] == 'Free To Play'].index] = 0
        df['price'] = df['price'].fillna(0)
    except:
        df['price'] = 0

    try:
        df['latest rating counter'] = df['latest rating counter'].str.replace(',','')
        df['latest rating counter'] = df['latest rating counter'].str.extract('(\d+)')
    except:
        df['latest rating counter'] = 0
    
    try:
        df['all rating counter'] = df['all rating counter'].str.replace(',','')
        df['all rating counter'] = df['all rating counter'].str.extract('(\d+)')
    except:
        df['all rating counter'] = 0
    #df['price'] = df['price'].astype(int)
    #df['latest rating counter'] = df['latest rating counter'].astype(int)
    #df['all rating counter'] = df['all rating counter'].astype(int)

    return df

second_data = create_game_info(second_list)
#second_data.to_csv('game_info_suv.csv')

<>:132: SyntaxWarning: invalid escape sequence '\d'
<>:138: SyntaxWarning: invalid escape sequence '\d'
<>:132: SyntaxWarning: invalid escape sequence '\d'
<>:138: SyntaxWarning: invalid escape sequence '\d'
C:\Users\GJ\AppData\Local\Temp\ipykernel_3812\1752941038.py:132: SyntaxWarning: invalid escape sequence '\d'
  df['latest rating counter'] = df['latest rating counter'].str.extract('(\d+)')
C:\Users\GJ\AppData\Local\Temp\ipykernel_3812\1752941038.py:138: SyntaxWarning: invalid escape sequence '\d'
  df['all rating counter'] = df['all rating counter'].str.extract('(\d+)')


1038250
게임가격 :  ₩ 61,000
모든평가 :  Mixed


C:\Users\GJ\AppData\Local\Temp\ipykernel_3812\1752941038.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'].iloc[df[df['price'] == 'Free'].index] = 0
C:\Users\GJ\AppData\Local\Temp\ipykernel_3812\1752941038.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'].iloc[df[df['price'] == 'Free To Play'].index] = 0


---
# 세일 합치기 코드

In [68]:
import pandas as pd

temp_df1 = pd.read_csv(r'static_data\이벤트 세일.csv')
temp_df2 = pd.read_csv(r'static_data\주간세일.csv')

# concat 

print(len(temp_df1))
print(len(temp_df2))

temp_df = pd.concat([temp_df1,temp_df2])


temp_df['year'] = temp_df['year'].astype(int)
temp_df = temp_df.sort_values('year',ascending=False)
temp_df = temp_df.reset_index()
temp_df.drop(columns=['index','Unnamed: 0'],inplace=True)
#temp_df
#temp_df.index
#temp_df.columns
temp_df.to_csv('전체세일.csv')


160
2069


---
# 가격변동 데이터

In [1]:
#
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np

from datetime import datetime

game_data_id = 10

def create_price_data(game_id):

    game_info_df = pd.read_csv(r'static_data\game_info.csv')
    game_info_df['AppID'].astype(int)
    origin_price = game_info_df[game_info_df['AppID'] == game_id]['price']

    print(origin_price)
    
    try:
        url = 'game_data\{}.xlsx'.format(game_id)
    except:
        print('{}Not xlsx File'.format(game_id))
        pass

    if url.split('.')[1] == 'csv':
        df = pd.read_csv(url)
    elif url.split('.')[1] == 'xlsx':
        df = pd.read_excel(url)
    else:
        return None

    df['Sale Start'] = np.nan
    df['Sale End'] = np.nan

    df['Origin Price'] = np.nan
    df['Origin Price'] = df['Origin Price'].fillna(int(origin_price.values[0])).astype(int)

    check_columns = []

    check_price_limit = 3

    if len(df) > check_price_limit - 1:
        for i in range(check_price_limit):
            check_columns.append(df['Final price'].iloc[0])
    else:
        for i in range(len(df)):
            check_columns.append(df['Final price'].iloc[0])

    print(check_columns)

    if len(df) > check_price_limit - 1:
        for i in range(check_price_limit,len(df)):
            check_columns.append(df['Final price'][i-3:i].max())


    print(check_columns, "/", len(df))

    df['Origin Price'] = check_columns

    df['Sale Rate'] = round(1 - (df['Final price']/df['Origin Price']),2)
    df['Sale Rate']
    #for item in df[df['Sale Rate'] < 0].index:
    #    df['Origin Price'].iloc[item] = df['Final price'].iloc[item]
    #    df['Sale Rate'].iloc[item] = 0

    df['Sale Start'] = df['DateTime']

    # + 0 세일이 없음
    # + 1 세일이 끝났음을 의미 
    index_data = df[df['Final price'] != df['Origin Price']].index

    if len(index_data) > 0:
        check_data_list = index_data if index_data[len(index_data) - 1] + 1 < (len(df) - 1) else index_data[:-1]
        df['Sale End'].iloc[check_data_list] = df['DateTime'].iloc[check_data_list + 1]

        df = df.dropna()
        df = df.reset_index()

        df.drop(columns=['index'],inplace=True)

        df['Sale Start'] = pd.to_datetime(df['Sale Start'])
        df['Sale End'] = pd.to_datetime(df['Sale End'])
    
        ready_to_sales = []
    
        for i in range(len(df) - 1):
            ready_to_sales.append(df['Sale Start'].iloc[i + 1] - df['Sale End'].iloc[i])
    
    
        ready_to_sales.append(pd.to_datetime(datetime.today()) - df['Sale Start'].iloc[-1])
    
        df['Ready to Sale'] = ready_to_sales
        df['Sale Period'] = (df['Sale End'] - df['Sale Start'])
    else:
        return df.dropna()
    
    return df

make_df = create_price_data(game_data_id)

make_df
#.to_csv('세일기간_가격_{}.csv'.format(game_id))



<>:21: SyntaxWarning: invalid escape sequence '\{'
<>:21: SyntaxWarning: invalid escape sequence '\{'
C:\Users\GJ\AppData\Local\Temp\ipykernel_9796\3459735622.py:21: SyntaxWarning: invalid escape sequence '\{'
  url = 'game_data\{}.xlsx'.format(game_id)


0    10500
Name: price, dtype: object


FileNotFoundError: [Errno 2] No such file or directory: 'game_data\\10.xlsx'

In [ ]:
import os
import re

csv_root_path = 'game_data'
csv_folder_list = os.listdir(csv_root_path)

file_list = []

for item in csv_folder_list:
    reg = re.compile('\d+')
    file = reg.findall(item)

    file_list.append(file[0])

create_game_info(file_list)


#create_price_data(game_data_id)

In [85]:



# 로그데이터를 만드는데


# 메인 - 세일주기

# 서브
# 세일 정보
# 게임 정보

# 값이 올랐을때
# 값이 내렸을 때



[63000,
 63000,
 63000,
 63000,
 63000,
 63000,
 63000,
 41580,
 27000,
 27000,
 27000,
 27000,
 27000,
 27000,
 27000,
 27000,
 27000,
 27000,
 27000,
 27000,
 27000,
 27000,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400,
 33400]

In [18]:
import re

total_gross = '50M'
total_net = '30M'
total_owners = '9.3M'

result_total_gross = 0
result_total_net = 0
result_total_owners = 0

if 'M' in total_gross:
    result_total_gross = float(total_gross.replace('M','')) * 1000000

if 'K' in total_gross:
    result_total_gross =float(total_gross.replace('K','')) * 1000

if 'M' in total_net:
    result_total_net = float(total_net.replace('M','')) * 1000000

if 'K' in total_net:
    result_total_net =float(total_net.replace('K','')) * 1000

if 'M' in total_owners:
    result_total_owners = float(total_owners.replace('M','')) * 1000000

if 'K' in total_owners:
    result_total_owners =float(total_owners.replace('K','')) * 1000
    
print(result_total_gross)
print(result_total_net)
print(result_total_owners)
#re.sub(r'[^0-9]', '',total_gross)


50000000.0
30000000.0
9300000.0
